In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
# install Python Reddit API Wrapper for getting external data Karma
%pip install pyspark==3.4.0
# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

import sparknlp
from sparknlp.base import *
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml.feature import CountVectorizer, StopWordsRemover,Tokenizer
from pyspark.ml import Pipeline

import plotly.graph_objects as go

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import numpy as np

from nltk.stem import WordNetLemmatizer
import nltk
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [4]:
spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.driver.memory","16G")
    .config("spark.driver.maxResultSize", "0") 
    .config("spark.kryoserializer.buffer.max", "2000M")
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")
    .config("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.ContainerCredentialsProvider")
    .getOrCreate()
)


:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0ef08427-1922-4d27-8e46-8b7ac23528d9;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

In [5]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
#s3_path = "s3a://sagemaker-us-east-1-038932893404/project/comments/yyyy=2021/part-00000-90796409-5783-4705-92c0-27c27eda8c4c-c000.snappy.parquet"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")
sampled_comments = comments.sample(withReplacement=False, fraction=0.1, seed=123)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-861795727138/project/comments/yyyy=*


23/11/20 06:58:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


shape of the comments dataframe is 1,053,280x21
CPU times: user 3.21 s, sys: 218 ms, total: 3.43 s
Wall time: 5min 37s


In [6]:
sampled_comments.cache()

DataFrame[author: string, author_cakeday: boolean, author_flair_css_class: string, author_flair_text: string, body: string, can_gild: boolean, controversiality: bigint, created_utc: timestamp, distinguished: string, edited: string, gilded: bigint, id: string, is_submitter: boolean, link_id: string, parent_id: string, permalink: string, retrieved_on: timestamp, score: bigint, stickied: boolean, subreddit: string, subreddit_id: string]

In [7]:
# Read data from EDA-Find top10 Creators in MakeupAddiction and list their names
sampled_topcreators= pd.read_csv('../../data/csv/sampled_creators_top10_total.csv')
sampled_topcreators= sampled_topcreators['author'].tolist()
sampled_topcreators = list(set(sampled_topcreators))

# Filter 10 top creators who comment most 
sampled_cmt_author= sampled_comments.select('author','body').filter(F.col('author').isin(sampled_topcreators))
sampled_top10_author_names = sampled_cmt_author.groupBy('author').count().orderBy(F.desc('count')).limit(10).select('author').rdd.flatMap(lambda x: x).collect()

## Sentiment Analysis

In [8]:
# Build the sentiment analysis pipeline
MODEL_NAME='sentimentdl_use_twitter'

documentAssembler = DocumentAssembler()\
    .setInputCol("cleaned_text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.
[ — ]

2023-11-20 07:09:45.791842: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

2023-11-20 07:09:50.493792: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-11-20 07:09:50.655419: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-11-20 07:09:50.844552: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-11-20 07:09:51.010390: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.


[ / ]

2023-11-20 07:09:51.424784: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.


[ | ]

[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
Download done! Loading the resource.
[OK!]


In [9]:
# Define a function to clean the text
def clean_text(data):
    cleaned_data = F.regexp_replace(data, '[^a-zA-Z0-9\s]', '')
    cleaned_data = F.lower(cleaned_data)
    cleaned_data = F.trim(cleaned_data)
    return cleaned_data


# Setup for sentiment analysis, organize the results
def stmt_analysis(a_name):
    sampled_cmt_author= sampled_comments.select('author','body').filter(F.col('author')==a_name)
    sampled_cmt_author = sampled_cmt_author.withColumn("cleaned_text", clean_text(F.col("body")))
    pipelineModel = nlpPipeline.fit(sampled_cmt_author)
    results = pipelineModel.transform(sampled_cmt_author)
    results= results.withColumn('sentiment',F.explode(results.sentiment.result))
    results= results.select('author','body','cleaned_text','sentiment')
    return results

# Perform sentiment analysis to each creator's comments 
analysis_collection= []
for i in range(len(sampled_top10_author_names)):
    author_name= sampled_top10_author_names[i]
    analysis_collection.append(stmt_analysis(author_name))

In [ ]:
# Aggregate sentiment analysis results for each author
top10Creator_stmt_analysis=pd.DataFrame()
results_collection= pd.DataFrame()

for i in range(len(sampled_top10_author_names)):
    analysis= analysis_collection[i].toPandas()
    top10Creator_stmt_analysis = pd.concat([top10Creator_stmt_analysis, analysis], ignore_index=True)  
    
    result= analysis_collection[i].groupby('author','sentiment').count().toPandas()
    results_collection = pd.concat([results_collection, result], ignore_index=True)  

## Word Count 


In [10]:
# Count word frequency
stmt_anal = spark.createDataFrame(top10Creator_stmt_analysis)


from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, ArrayType, IntegerType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
import pyspark.sql.functions as F
# Tokenization
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol='cleaned_text', outputCol='words')
tokenized = tokenizer.transform(stmt_anal)

# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='filtered_words')
cleaned_data = remover.transform(tokenized)


from nltk.stem import WordNetLemmatizer
import nltk
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

# Define a UDF for the lemmatization
lemmatize_udf = udf(lemmatize_words, ArrayType(StringType()))

# Apply lemmatization to the tokenized words
lemmatized_data = cleaned_data.withColumn("lemmatized_words", lemmatize_udf(F.col("filtered_words")))

# Hashing TF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures")
featurized_data = hashingTF.transform(lemmatized_data)

# IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(featurized_data)
tfidf_data = idf_model.transform(featurized_data)

from pyspark.ml.feature import CountVectorizer

# Tokenization and removing stop words should be done before this step

# Fit a CountVectorizer
cv = CountVectorizer(inputCol="filtered_words", outputCol="cv_features")
cv_model = cv.fit(cleaned_data)

# Now, we can access the vocabulary
vocabulary = cv_model.vocabulary

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import numpy as np

def extract_top_terms_from_vector(vector, vocab, num_top_terms=10):
    # Convert sparse vector to dense
    dense_vector = vector.toArray()
    
    # Get indices of the top terms
    top_indices = np.argsort(dense_vector)[-num_top_terms:][::-1]
    
    # Map indices to words using the vocabulary
    top_terms = [vocab[i] for i in top_indices if i < len(vocab) and dense_vector[i] > 0]
    
    return top_terms

extract_top_terms_udf = udf(lambda vector: extract_top_terms_from_vector(vector, vocabulary), ArrayType(StringType()))


# Apply the UDF to your DataFrame
tfidf_with_top_terms = tfidf_data.withColumn("top_terms", extract_top_terms_udf("features"))

from pyspark.sql.functions import explode, count, col, row_number
from pyspark.sql.window import Window

# Explode the top_terms column to count each term individually
exploded_terms = tfidf_with_top_terms.select('top_terms').withColumn('term', explode('top_terms'))
term_counts = exploded_terms.groupBy('term').agg(count('term').alias('term_count')).sort('term_count',ascending=False)

term_counts.show()

NameError: name 'top10Creator_stmt_analysis' is not defined

## Visualizaion

In [ ]:
# save the analytical results 
top10Creator_stmt_analysis.to_csv('../../data/csv/top10Creator_stmt_analysis.csv')
results_collection.to_csv('../../data/csv/top10Creator_stmt_results.csv')

In [ ]:
# Plot figure
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

fig = go.Figure()

colors = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}

for sentiment, color in colors.items():
    subset = results_collection[results_collection['sentiment'] == sentiment]
    fig.add_trace(go.Bar(
        x=subset['author'],
        y=subset['count'],
        name=sentiment.title(), 
        marker_color=color))

fig.update_layout(
    title="Sentiment Analysis Results by Top20 Creators' comments",
    xaxis_title='Author',
    yaxis_title='Count',
    barmode='stack',
    legend=dict(title='Sentiment'),
    hovermode='x')

fig.write_html('../../data/plots/Top10Creators_stmt_results.html')

fig.show()

* REVISED-Business Goals
 
   Following the exploratory task of identifying the top 10 creators, we move on to assess the sentiment of top creators' comments to gauge the overall emotional tone of their response. This will provide further and more scientific insights into the influencer-audience interactions in MakeupAddiction.

  
* Technical Proposal

    Utilize NLP libraries like NLTK, and preprocess the text data, including tokenization, removing stop words, and lemmatization for word frequency analysis. Apply sentiment analysis to the preprocessed comments to determine their emotional tone. Calculate sentiment scores for each comment to quantify the intensity of emotions. Aggregate the sentiment scores of comments from each influencer. Visualize the sentiment analysis results using charts to provide a clear understanding of influencers' emotional impact. 
    
We have refined our business goal after a thorough examination of the data, as we encountered challenges linking comments to their respective submissions. Despite attempts to use inner joins such as submission.id == comments.link_id and submission.id == comments.parent_id, we were unable to establish the necessary connections.
In light of these challenges in investigating user comments directed at influencers, we have pivoted our focus to analyzing influencers' responses to their audience. This approach allows us to explore the dynamic interactions and reactions between influencers and their audience, providing valuable insights into the engagement dynamics within our platform.







Analysis Report

1. **Authors and Sentiments:**
   - The table includes 10 authors such as ahbagelxo, waterfromthevalley, Ralucamakeup. They have been selected as part of the EDA task aimed at identifying the Top 10 Creators from 2021 to 2023, and they are the influencers who respond the most to their comments.
   - Sentiments are categorized as positive, negative, or neutral.

2. **Sentiment Summary:**
   - The total count of positive sentiments across all authors is 586. This represents approximately 78% of the total sentiments.
   - The total count of neutral sentiments is 16, constituting around 2.4% of the total sentiments.
   - The total count of negative sentiments is 54, accounting for approximately 11.3% of the total sentiments.

3. **Overall Positivity Across Authors:**
   - The percentage of positive sentiments is notably high across most authors, ranging from 72% to 92%. While positive sentiments prevail, there is notable variation in the distribution. Some influencers, like ahbagelxo and waterfromthevalley, have a higher percentage of positive sentiments, while others, like Sick-Sad-World32, have a more balanced distribution.
   - Neutral and negative sentiments, individually, make up a smaller portion of the total sentiments. Neutral sentiments range from 1% to 4%, and negative sentiments range from 1% to 3% across different authors.

- The analysis provides insights into how each influencer communicates with their audience, which can influence their online presence and relationship with followers.


4. **User Feedback and Expectations:**
   - The public gains insights into how influencers engage with their audience. Understanding the sentiments of influencer responses provides a glimpse into how these figures interact with their followers and manage feedback.

5. **Influencer-Audience Relationship:**
   - Users can better understand the dynamics of the relationship between influencers and their audience. Positive interactions may strengthen this relationship, while negative or neutral responses may impact how followers perceive an influencer.

6. **Influencer Reputation:**
   - Users can form opinions about an influencer's reputation based on their responses to comments. Positive and respectful engagement may enhance an influencer's reputation, while negative or dismissive interactions could have the opposite effect.

- In summary, this NLP task contributes to a more informed and engaged online community by providing insights into influencer-audience interactions. Users can make more informed decisions about who to follow, and influencers may gain a better understanding of their impact on their audience.
